In [1]:
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import config


In [2]:
def parse_jssp_file(filename):
    jobs_data = []

    with open(filename, "r") as f:
        # Enlever les lignes de commentaires
        lines = [
            line.strip()
            for line in f
            if line.strip() and not line.strip().startswith("#")
        ]

    # Première ligne : nb jobs, nb machines
    num_jobs, num_machines = map(int, lines[0].split())

    # Lignes suivantes : jobs
    for i in range(1, num_jobs + 1):
        data = list(map(int, lines[i].split()))
        job = []

        for k in range(0, len(data), 2):
            machine = data[k]
            duration = data[k + 1]
            job.append((machine, duration))

        jobs_data.append(job)

    return jobs_data, num_jobs, num_machines

jobs_data, num_jobs, num_machines = parse_jssp_file(config.INSTANCE_DIR + "/ft06.txt")

In [ ]:
from ortools.linear_solver import pywraplp

def solve_milp_jsp(jobs_data, num_jobs, num_machines, time_limit_sec=300):
    # Création du solveur (SCIP est bon pour le mixte-entier)
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("SCIP solver not found.")
        return

    # Calcul de l'horizon (Borne M pour le Big-M)
    horizon = sum(task[1] for job in jobs_data for task in job)
    print(f"Horizon (Big-M): {horizon}")

    # --- 1. Variables ---

    # starts[j][t] : temps de début de la t-ème tâche du job j
    starts = {}
    for j in range(num_jobs):
        for t in range(num_machines): # Supposant que chaque job a 'num_machines' tâches
            starts[(j, t)] = solver.IntVar(0.0, float(horizon), f'start_{j}_{t}')

    # makespan : variable à minimiser
    makespan = solver.IntVar(0.0, float(horizon), 'makespan')

    # Dictionnaire helper pour retrouver quelle tâche correspond à quelle machine
    # task_on_machine[m] = liste de tuples (job_index, task_index_in_job, duration)
    task_on_machine = {m: [] for m in range(num_machines)}

    for j in range(num_jobs):
        for t, (mach, dur) in enumerate(jobs_data[j]):
            task_on_machine[mach].append((j, t, dur))

    # --- 2. Contraintes ---

    print("Création des contraintes de précédence...")
    # A. Contraintes de précédence (Intra-Job)
    # start[j][t+1] >= start[j][t] + duration[j][t]
    for j in range(num_jobs):
        for t in range(len(jobs_data[j]) - 1):
            dur = jobs_data[j][t][1]
            solver.Add(starts[(j, t + 1)] >= starts[(j, t)] + dur)

        # Contrainte pour le Makespan (doit être après la dernière tâche de chaque job)
        last_t = len(jobs_data[j]) - 1
        last_dur = jobs_data[j][last_t][1]
        solver.Add(makespan >= starts[(j, last_t)] + last_dur)

    print("Création des contraintes disjonctives (Big-M)...")
    # B. Contraintes Disjonctives (Inter-Job sur même machine)
    # Pour chaque machine, et pour toute paire de tâches (i, j) sur cette machine :
    count_disj = 0
    M = float(horizon) # Big-M assez grand

    for m in range(num_machines):
        tasks = task_on_machine[m]
        # Comparaison paire à paire unique
        for idx1 in range(len(tasks)):
            for idx2 in range(idx1 + 1, len(tasks)):
                j1, t1, dur1 = tasks[idx1]
                j2, t2, dur2 = tasks[idx2]

                # Variable binaire Y : 1 si j1 passe AVANT j2, 0 sinon
                y = solver.IntVar(0, 1, f'y_m{m}_j{j1}_j{j2}')

                # Big-M Transformation:
                # Si y=1 (j1 avant j2) :
                #   Eq 1 devient: start2 >= start1 + dur1 - 0   (Active)
                #   Eq 2 devient: start1 >= start2 + dur2 - M   (Inutile car toujours vraie)

                # Si j1 avant j2 => start2 >= start1 + dur1
                solver.Add(starts[(j2, t2)] >= starts[(j1, t1)] + dur1 - M * (1 - y))

                # Si j2 avant j1 => start1 >= start2 + dur2
                solver.Add(starts[(j1, t1)] >= starts[(j2, t2)] + dur2 - M * y)

                count_disj += 1

    print(f"Nombre de paires disjonctives (variables binaires) : {count_disj}")

    # --- 3. Objectif et Résolution ---
    solver.Minimize(makespan)

    # Paramètres du solveur
    solver.SetTimeLimit(time_limit_sec * 1000)

    print(f"Lancement du solveur SCIP (Time limit: {time_limit_sec}s)...")
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print("\nSolution trouvée !")
        print(f"Makespan: {solver.Objective().Value()}")
        if status == pywraplp.Solver.OPTIMAL:
            print("(Optimalité prouvée)")
        else:
            print("(Solution faisable, potentiellement non optimale)")
    else:
        print("Pas de solution trouvée dans le temps imparti.")

# --- Test ---
jobs_data, num_jobs, num_machines = parse_jssp_file(config.INSTANCE_DIR + "/ft10.txt")
solve_milp_jsp(jobs_data, num_jobs, num_machines)

Horizon (Big-M): 5109
Création des contraintes de précédence...
Création des contraintes disjonctives (Big-M)...
Nombre de paires disjonctives (variables binaires) : 450
Lancement du solveur SCIP (Time limit: 300s)...

Solution trouvée !
Makespan: 941.0
(Solution faisable, potentiellement non optimale)


# Résultat de l'approche MILP (temps limite 10 min)

## test sur abz5.txt (opti 1234):
### Temps soumis 10 min

Horizon (Big-M): 7773
Création des contraintes de précédence...
Création des contraintes disjonctives (Big-M)...
Nombre de paires disjonctives (variables binaires) : 450
Lancement du solveur SCIP (Time limit: 600s)...

Solution trouvée !
Makespan: 1239.0

## test sur ft06.txt (opti 55) :
### Temps soumis 127 ms
Horizon (Big-M): 197
Création des contraintes de précédence...
Création des contraintes disjonctives (Big-M)...
Nombre de paires disjonctives (variables binaires) : 90
Lancement du solveur SCIP (Time limit: 600s)...

Solution trouvée !
Makespan: 55.0
(Optimalité prouvée)


# Résultat

55.0 est bien l'optimal connu et prouvé de l'instance ft06. Ton code est donc mathématiquement juste.

127 ms vs Timeout sur abz5 : C'est la démonstration parfaite de l'explosion combinatoire (NP-Hard).

ft06 (6x6) = 36 tâches.

abz5 (10x10) = 100 tâches.

Passer de 36 à 100 tâches ne multiplie pas le temps par 3, mais par un facteur exponentiel (des milliards de fois plus dur).



# https://medium.com/data-science/a-comprehensive-guide-to-modeling-techniques-in-mixed-integer-linear-programming-3e96cc1bc03d